In [1]:
import sys
print(sys.executable)

/home/codespace/.local/share/virtualenvs/web-service-mlflow-huJd73E5/bin/python


In [2]:
import sklearn
sklearn.__version__

'1.0.2'

In [3]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [4]:
from sklearn.pipeline import make_pipeline

In [5]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='file:///workspaces/MLOps/04-deployment/web-service-mlflow/mlflow_artifacts/1', creation_time=1769513486389, experiment_id='1', last_update_time=1769513486389, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [6]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [7]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [8]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val) ** 0.5
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

2026/01/27 12:00:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2026/01/27 12:00:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run spiffy-eel-834 at: http://127.0.0.1:5000/#/experiments/1/runs/51b28eb5ef9047e3ab620e58530077cf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [9]:
from mlflow.tracking import MlflowClient

In [10]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '51b28eb5ef9047e3ab620e58530077cf'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
path = client.download_artifacts(run_id=RUN_ID, path='model')
print("Downloaded to:", path)

Downloaded to: /tmp/tmpeh27b5ly/model/
